In [15]:
import numpy as np
import cv2
from PIL import Image
from glob import glob
from string import ascii_lowercase, digits
import random
import json
import cv2

import pytesseract
from pytesseract import image_to_string

pytesseract.pytesseract.tesseract_cmd = "C:/Program Files/Tesseract-OCR/tesseract"
TESSDATA_PREFIX = r"C:\Program Files\Tesseract-OCR"

max_enhance_level = True

screenshot_path = 'screenshots/'

In [16]:
def process(k,img):
    if  k == 'plus' or k == 'level':
        thresh = cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU
        thresh_low = 81
        proc = cv2.cvtColor(cv2.medianBlur(cv2.threshold(cv2.cvtColor(cv2.resize(img, (0, 0), fx=5, fy=5), cv2.COLOR_RGB2GRAY), thresh_low, 255, thresh)[1], 3), cv2.COLOR_GRAY2RGB)
        data = image_to_string(Image.fromarray(proc), lang='eng', config='--psm 7').replace('+b','6').replace('>','0')
        if not any(i.isdigit() for i in data):
            thresh_low = 100
            proc = cv2.cvtColor(cv2.medianBlur(cv2.threshold(cv2.cvtColor(cv2.resize(img, (0, 0), fx=5, fy=5), cv2.COLOR_RGB2GRAY), thresh_low, 255, thresh)[1], 3), cv2.COLOR_GRAY2RGB)
            data = image_to_string(Image.fromarray(proc), lang='eng', config='--psm 7').replace('+b','6').replace('>','0')
            if not any(i.isdigit() for i in data):
                thresh_low = 125
                proc = cv2.cvtColor(cv2.medianBlur(cv2.threshold(cv2.cvtColor(cv2.resize(img, (0, 0), fx=5, fy=5), cv2.COLOR_RGB2GRAY), thresh_low, 255, thresh)[1], 3), cv2.COLOR_GRAY2RGB)
                data = image_to_string(Image.fromarray(proc), lang='eng', config='--psm 7').replace('+b','6').replace('>','0')
                if not any(i.isdigit() for i in data):
                    data = str(0)
    else:
        thresh = cv2.THRESH_BINARY_INV
        thresh_low = 70
        proc = cv2.cvtColor(cv2.medianBlur(cv2.threshold(cv2.cvtColor(cv2.resize(img, (0, 0), fx=5, fy=5), cv2.COLOR_BGR2GRAY), thresh_low, 255, thresh)[1], 3), cv2.COLOR_GRAY2RGB)
        data = image_to_string(Image.fromarray(proc), lang='eng', config='--psm 6').replace('Rina','Ring').replace('Edic','Epic').replace('Enic','Epic')
    return data

def stat_converter(stat):
    result = ''
    if 'attack' in stat.lower():
        result = 'Atk'
        if '%' in stat:
            result += 'P'
    if 'health' in stat.lower():
        result = 'HP'
        if '%' in stat:
            result += 'P'
    if 'defense' in stat.lower():
        result = 'Def'
        if '%' in stat:
            result += 'P'
    if 'speed' in stat.lower():
        result = 'Spd'
    if 'chance' in stat.lower():
        result = 'CChance'
    if 'damage' in stat.lower():
        result = 'CDmg'
    if 'effectiveness' in stat.lower():
        result = 'Eff'
    if 'resistance' in stat.lower():
        result = 'Res'
    return result
  
def digit_filter(val):
    if not val:
        return 0
    elif val == '.':
        return 0
    else:
        return int(''.join(filter(str.isdigit,val)))

def char_filter(val):
    return ''.join(filter(str.isalpha,val)).capitalize()

def max_stat(data,item):
    stat = stat_converter(data)
    val = digit_filter(data)
    if item['ability'] < 15:
        if item['level'] in range(58,73):
            if stat == 'CChance': val = 45
            elif stat == 'CDmg': val = 55
            elif stat == 'Spd': val = 35
            elif item['slot'] == ('Necklace' or 'Ring' or 'Boots'): val = 50
            elif stat == 'HP': val = 2295
            elif stat == 'Def': val = 250 
            elif stat == 'Atk': val = 425 
        elif item['level'] in range(74,86):
            if stat == 'CChance': val = 55
            elif stat == 'CDmg': val = 65
            elif stat == 'Spd': val = 40
            elif item['slot'] == ('Necklace' or 'Ring' or 'Boots'): val = 60
            elif stat == 'HP': val = 2700
            elif stat == 'Def': val = 300
            elif stat == 'Atk': val = 500
        elif item['level'] in range(87,89):
            if stat == 'CChance': val = 60
            elif stat == 'CDmg': val = 70
            elif stat == 'Spd': val = 45
            elif item['slot'] == ('Necklace' or 'Ring' or 'Boots'): val = 65
            elif stat == 'HP': val = 2765
            elif stat == 'Def': val = 310
            elif stat == 'Atk': val = 515
    return val

In [17]:
export = {'processVersion':'1','heroes':[],'items':[]}
temp_list = []

print('Beginning export process...')

filenames = glob(screenshot_path+'*.png')
filenames.extend(glob(screenshot_path+'*.jpg'))
filenames.extend(glob(screenshot_path+'*.jpeg'))

did_break = False
errors = 0
for n,name in enumerate(filenames):
    
    print('currently processing '+name.replace('screenshots\\',''))
    img = cv2.imread(name)
    if img.shape[0] != 1080 or img.shape[1] != 2200:
        print('ERROR: This screenshots is not 2200x1080!')
        print('Please retake the screenshot with a resolution of 2200x1080')
        did_break = True
        break
    
    top_decoration = cv2.imread('e7/top.jpg',0)
    a, b, c, max_loc = cv2.minMaxLoc(cv2.matchTemplate(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), top_decoration, cv2.TM_CCOEFF_NORMED))
    top_box = img[max_loc[1]:max_loc[1]+160,740:1190] 

    bottom_decoration = cv2.imread('e7/bottom.jpg',0)
    a, b, c, max_loc = cv2.minMaxLoc(cv2.matchTemplate(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), bottom_decoration, cv2.TM_CCOEFF_NORMED))
    bottom_box = img[max_loc[1]+25:max_loc[1]+360,740:1190]

    item = {'locked':False,'efficiency':0}

    top_coords = {'type': [[20,70],[172,432]],
                  'level': [[19,44],[37,66]],
                  'plus': [[11,34],[139,168]]}
    
    for k in top_coords.keys():
        data = process(k,top_box[top_coords[k][0][0]:top_coords[k][0][1],top_coords[k][1][0]:top_coords[k][1][1]])
        if k == 'type':
            item['rarity'] = char_filter(data.split(' ')[0])
            item['slot'] = char_filter(data.split(' ')[1].split('\n')[0])
            if len(item['rarity']) == 0:
                print('Error: no rarity detected')
            if len(item['slot']) == 0:
                print('Error: no slot detected')
        if k == 'level':
            item['level'] = digit_filter(data.replace('S','5').replace('B','8').replace('a','8'))
            if item['level'] == 0:
                item['level'] = 1
        if k == 'plus':
            item['ability'] = digit_filter(data.replace('S','5').replace('B','8').replace('a','8'))

    bot_coords = {'main': [[8,70],[65,435]],
                  'subs': [[98,255],[25,435]],
                  'set': [[280,340],[76,435]]}
    
    for k in bot_coords.keys():
        data = process(k,bottom_box[bot_coords[k][0][0]:bot_coords[k][0][1],bot_coords[k][1][0]:bot_coords[k][1][1]])
        if k == 'main':
            stat = stat_converter(data)
            if max_enhance_level is True:
                item['mainStat'] = [stat,max_stat(data,item)]
            else:
                item['mainStat'] = [stat,digit_filter(data)]     
        
        if k == 'subs':
            for n,entry in enumerate(data.split('\n')):
                if n < 4:
                    stat = stat_converter(entry)
                    val = digit_filter(entry.replace('T','7'))
                    if len(stat) > 0 and val != 0:
                        item['subStat'+str(n+1)] = [stat,val]
        if k == 'set':
            item['set'] = char_filter(data.split(' Set')[0])
            if len(item['set']) == 0:
                print('Error: no set detected')
    
    if item not in temp_list and len(item['slot']) > 0 and len(item['rarity']) > 0:
        temp_list.append(item)
        item['id'] = 'jt'+''.join(random.choice(digits+ascii_lowercase) for _ in range(6))
        export['items'].append(item)
        print(len(export['items']),"exported out of",len(filenames)-errors,'valid item%s \n' % ('s' if len(filenames)-errors>1 else ''))
    else:
        errors += 1
        print('Item not exported, a stat was read wrong somewhere \n')
    
if not did_break:
    with open('exported_gear.json', 'w') as f:
        json.dump(export, f)
    print('\nExporting complete!')

Beginning export process...
currently processing Screenshot_2020-10-25-22-50-43.png
1 exported out of 1 valid item 

Exporting complete!
